# LLAMATOR MCP Server — cURL Tutorial

This notebook demonstrates how to call:
- the HTTP API
- the MCP Streamable HTTP endpoint (JSON-RPC)

All examples use `curl` and assume the service is already running.

## 0) Setup

The cell below reads common settings from environment variables.

Supported variables:
- `LLAMATOR_MCP_BASE_URL` (optional)
- `LLAMATOR_MCP_HTTP_PUBLIC_PORT` / `LLAMATOR_MCP_HTTP_PORT` (optional)
- `LLAMATOR_MCP_API_KEY` (optional)
- `LLAMATOR_MCP_MCP_MOUNT_PATH` (optional)
- `LLAMATOR_MCP_TEST_MCP_PROTOCOL_VERSION` (optional)

If you have a local `.env`, it will be loaded automatically.

In [ ]:
from __future__ import annotations

import json
import os
import subprocess
import time
import urllib.parse

from dotenv import load_dotenv

# Load env from a local .env file (if present).
load_dotenv()

HTTP_PUBLIC_PORT: str = os.getenv("LLAMATOR_MCP_HTTP_PUBLIC_PORT", "").strip()
HTTP_PORT: str = os.getenv("LLAMATOR_MCP_HTTP_PORT", "").strip()
PORT: str = HTTP_PUBLIC_PORT or HTTP_PORT or "8000"

BASE_URL: str = os.getenv("LLAMATOR_MCP_BASE_URL", f"http://localhost:{PORT}").strip().rstrip("/")

API_KEY: str = os.getenv("LLAMATOR_MCP_API_KEY", "").strip()
AUTH_HEADER_ARGS: list[str] = ["-H", f"X-API-Key: {API_KEY}"] if API_KEY else []

MCP_MOUNT_PATH: str = os.getenv("LLAMATOR_MCP_MCP_MOUNT_PATH", "/mcp").strip() or "/mcp"
MCP_ENDPOINT: str = f"{BASE_URL}{MCP_MOUNT_PATH.rstrip('/')}/"

MCP_PROTOCOL_VERSION: str = os.getenv("LLAMATOR_MCP_TEST_MCP_PROTOCOL_VERSION", "2025-03-26").strip()

print(f"Base URL: {BASE_URL}")
print(f"MCP endpoint: {MCP_ENDPOINT}")
print(f"API key enabled: {bool(API_KEY)}")
print(f"MCP protocol version: {MCP_PROTOCOL_VERSION}")


## 1) HTTP API

### 1.1 Healthcheck

This endpoint is useful to confirm the server is reachable.

In [ ]:
cmd: list[str] = [
    "curl",
    "-sS",
    "-i",
    "-X",
    "GET",
    f"{BASE_URL}/v1/health",
    "-H",
    "Accept: application/json",
    *AUTH_HEADER_ARGS,
]

print("Running:", " ".join(cmd))
print(subprocess.check_output(cmd, text=True))


### 1.2 Create a run (preset)

A run request contains:
- `tested_model`: what you want to test
- `plan`: which tests to run

This example uses `plan.preset_name`.

In [ ]:
payload_preset: dict[str, object] = {
    "tested_model": {
        "kind": "openai",
        "base_url": os.getenv("LLAMATOR_MCP_TEST_TESTED_BASE_URL", "http://host.docker.internal:1234/v1").strip(),
        "model": os.getenv("LLAMATOR_MCP_TEST_TESTED_MODEL", "llm").strip(),
        "api_key": os.getenv("LLAMATOR_MCP_TEST_TESTED_API_KEY", "lm-studio").strip() or None,
        "temperature": 0.2,
        "system_prompts": ["You are a helpful assistant."],
        "model_description": "Example tested model",
    },
    "run_config": {
        "enable_reports": False,
    },
    "plan": {
        "preset_name": "owasp:llm10",
        "num_threads": 1,
    },
}

# Remove null api_key to keep the request clean.
tested_model_obj: dict[str, object] = payload_preset["tested_model"]  # type: ignore[assignment]
if tested_model_obj.get("api_key") is None:
    tested_model_obj.pop("api_key", None)

print("Request payload:")
print(json.dumps(payload_preset, ensure_ascii=False, indent=2, sort_keys=True))


In [ ]:
# Send the request with curl, passing JSON via stdin (no temporary files).
payload_text: str = json.dumps(payload_preset, ensure_ascii=False)

cmd: list[str] = [
    "curl",
    "-sS",
    "-X",
    "POST",
    f"{BASE_URL}/v1/tests/runs",
    "-H",
    "Accept: application/json",
    "-H",
    "Content-Type: application/json",
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
out: str = subprocess.check_output(cmd, input=payload_text, text=True)

created_preset: object = json.loads(out)
if not isinstance(created_preset, dict):
    raise ValueError("Expected a JSON object from /v1/tests/runs.")

print("Response:")
print(json.dumps(created_preset, ensure_ascii=False, indent=2, sort_keys=True))

JOB_ID_PRESET: str = str(created_preset.get("job_id", "")).strip()
if not JOB_ID_PRESET:
    raise ValueError("Response did not contain job_id.")

print(f"Created job_id: {JOB_ID_PRESET}")


### 1.3 Poll job status (preset)

This section demonstrates a simple polling loop:
- request `/v1/tests/runs/{job_id}`
- wait until the job becomes `succeeded` or `failed`

The final response contains `result` and/or `error` fields.

In [ ]:
job_id: str = JOB_ID_PRESET
timeout_s: float = 3600.0
poll_interval_s: float = 0.5

deadline: float = time.time() + timeout_s
last_status: str | None = None
final_job: dict[str, object] | None = None

while time.time() < deadline:
    cmd: list[str] = [
        "curl",
        "-sS",
        "-X",
        "GET",
        f"{BASE_URL}/v1/tests/runs/{job_id}",
        "-H",
        "Accept: application/json",
        *AUTH_HEADER_ARGS,
    ]

    out: str = subprocess.check_output(cmd, text=True)
    payload_any: object = json.loads(out)
    if not isinstance(payload_any, dict):
        raise ValueError("Expected a JSON object from /v1/tests/runs/{job_id}.")

    status: str = str(payload_any.get("status", "")).strip().lower()
    updated_at: str = str(payload_any.get("updated_at", "")).strip()

    if status != last_status:
        print(f"Status changed: {status} (updated_at={updated_at})")
        last_status = status

    if status in ("succeeded", "failed"):
        final_job = payload_any
        break

    time.sleep(poll_interval_s)

if final_job is None:
    raise TimeoutError(f"Job did not finish within {timeout_s} seconds: job_id={job_id}")

print("Final job response:")
print(json.dumps(final_job, ensure_ascii=False, indent=2, sort_keys=True))


### 1.4 List artifacts (preset)

Artifacts are files produced by a run. The API returns metadata for each file.

In [ ]:
job_id: str = JOB_ID_PRESET

cmd: list[str] = [
    "curl",
    "-sS",
    "-X",
    "GET",
    f"{BASE_URL}/v1/tests/runs/{job_id}/artifacts",
    "-H",
    "Accept: application/json",
    *AUTH_HEADER_ARGS,
]

print("Running:", " ".join(cmd))
out: str = subprocess.check_output(cmd, text=True)

artifacts_list: object = json.loads(out)
if not isinstance(artifacts_list, dict):
    raise ValueError("Expected a JSON object from /artifacts.")

print("Artifacts list response:")
print(json.dumps(artifacts_list, ensure_ascii=False, indent=2, sort_keys=True))


### 1.5 Resolve a download link for the first artifact (preset)

The server returns a JSON object with a temporary `download_url`.

In [ ]:
job_id: str = JOB_ID_PRESET

files_val: object = artifacts_list.get("files") if isinstance(artifacts_list, dict) else None
files: list[dict[str, object]] = [x for x in files_val if isinstance(x, dict)] if isinstance(files_val, list) else []

if not files:
    print("No artifacts were produced for this job.")
else:
    first_path: str = str(files[0].get("path", "")).strip()
    if not first_path:
        print("The first artifact entry does not contain a valid path.")
        print(json.dumps(files[0], ensure_ascii=False, indent=2, sort_keys=True))
    else:
        quoted: str = urllib.parse.quote(first_path, safe="/")
        url: str = f"{BASE_URL}/v1/tests/runs/{job_id}/artifacts/{quoted}"

        cmd: list[str] = [
            "curl",
            "-sS",
            "-X",
            "GET",
            url,
            "-H",
            "Accept: application/json",
            *AUTH_HEADER_ARGS,
        ]

        print("Running:", " ".join(cmd))
        out: str = subprocess.check_output(cmd, text=True)

        payload: object = json.loads(out)
        if not isinstance(payload, dict):
            raise ValueError("Expected a JSON object from the artifact download resolver.")

        print("Download link response:")
        print(json.dumps(payload, ensure_ascii=False, indent=2, sort_keys=True))


### 1.6 Create a run (explicit basic tests)

Instead of a preset, you can pass `plan.basic_tests`.

Each test has:
- `code_name`
- `params`: a list of `{name, value}` objects

In [ ]:
basic_tests: list[tuple[str, dict[str, object]]] = [
    ("repetition_token", {"num_attempts": 1, "repeat_count": 3}),
    ("system_prompt_leakage", {"custom_dataset": None, "multistage_depth": 3, "num_attempts": 1}),
]

payload_basic_tests: dict[str, object] = {
    "tested_model": {
        "kind": "openai",
        "base_url": os.getenv("LLAMATOR_MCP_TEST_TESTED_BASE_URL", "http://host.docker.internal:1234/v1").strip(),
        "model": os.getenv("LLAMATOR_MCP_TEST_TESTED_MODEL", "llm").strip(),
        "api_key": os.getenv("LLAMATOR_MCP_TEST_TESTED_API_KEY", "lm-studio").strip() or None,
        "temperature": 0.3,
        "system_prompts": ["You are a helpful assistant."],
        "model_description": "Example tested model",
    },
    "run_config": {
        "enable_reports": False,
    },
    "plan": {
        "num_threads": 1,
        "basic_tests": [
            {
                "code_name": code_name,
                "params": [{"name": k, "value": v} for k, v in params.items()],
            }
            for code_name, params in basic_tests
        ],
    },
}

# Remove null api_key to keep the request clean.
tested_model_obj2: dict[str, object] = payload_basic_tests["tested_model"]  # type: ignore[assignment]
if tested_model_obj2.get("api_key") is None:
    tested_model_obj2.pop("api_key", None)

print("Request payload:")
print(json.dumps(payload_basic_tests, ensure_ascii=False, indent=2, sort_keys=True))


In [ ]:
payload_text: str = json.dumps(payload_basic_tests, ensure_ascii=False)

cmd: list[str] = [
    "curl",
    "-sS",
    "-X",
    "POST",
    f"{BASE_URL}/v1/tests/runs",
    "-H",
    "Accept: application/json",
    "-H",
    "Content-Type: application/json",
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
out: str = subprocess.check_output(cmd, input=payload_text, text=True)

created_basic: object = json.loads(out)
if not isinstance(created_basic, dict):
    raise ValueError("Expected a JSON object from /v1/tests/runs.")

print("Response:")
print(json.dumps(created_basic, ensure_ascii=False, indent=2, sort_keys=True))

JOB_ID_BASIC: str = str(created_basic.get("job_id", "")).strip()
if not JOB_ID_BASIC:
    raise ValueError("Response did not contain job_id.")

print(f"Created job_id: {JOB_ID_BASIC}")


### 1.7 Poll job status (basic tests)

Same polling logic as the preset run.

In [ ]:
job_id: str = JOB_ID_BASIC
timeout_s: float = 3600.0
poll_interval_s: float = 0.5

deadline: float = time.time() + timeout_s
last_status: str | None = None
final_job_basic: dict[str, object] | None = None

while time.time() < deadline:
    cmd: list[str] = [
        "curl",
        "-sS",
        "-X",
        "GET",
        f"{BASE_URL}/v1/tests/runs/{job_id}",
        "-H",
        "Accept: application/json",
        *AUTH_HEADER_ARGS,
    ]

    out: str = subprocess.check_output(cmd, text=True)
    payload_any: object = json.loads(out)
    if not isinstance(payload_any, dict):
        raise ValueError("Expected a JSON object from /v1/tests/runs/{job_id}.")

    status: str = str(payload_any.get("status", "")).strip().lower()
    updated_at: str = str(payload_any.get("updated_at", "")).strip()

    if status != last_status:
        print(f"Status changed: {status} (updated_at={updated_at})")
        last_status = status

    if status in ("succeeded", "failed"):
        final_job_basic = payload_any
        break

    time.sleep(poll_interval_s)

if final_job_basic is None:
    raise TimeoutError(f"Job did not finish within {timeout_s} seconds: job_id={job_id}")

print("Final job response:")
print(json.dumps(final_job_basic, ensure_ascii=False, indent=2, sort_keys=True))


## 2) MCP (Streamable HTTP JSON-RPC)

The MCP endpoint uses JSON-RPC over HTTP POST.

Typical sequence:
1. `initialize`
2. `notifications/initialized`
3. `tools/list`
4. `tools/call`

### 2.1 Initialize

This call may return `Mcp-Session-Id` in response headers. If present, include it in subsequent calls.

In [ ]:
init_msg: dict[str, object] = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": MCP_PROTOCOL_VERSION,
        "capabilities": {},
        "clientInfo": {"name": "curl-notebook", "version": "1.0.0"},
    },
}

payload_text: str = json.dumps(init_msg, ensure_ascii=False)

cmd: list[str] = [
    "curl",
    "-sS",
    "-i",
    "-X",
    "POST",
    MCP_ENDPOINT,
    "-H",
    "Accept: application/json, text/event-stream",
    "-H",
    "Content-Type: application/json",
    "-H",
    f"MCP-Protocol-Version: {MCP_PROTOCOL_VERSION}",
    "-H",
    f"Origin: {BASE_URL}",
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
raw: str = subprocess.check_output(cmd, input=payload_text, text=True)

print("Raw response (headers + body):")
print(raw[:4000])

header_block: str = raw.split("\r\n\r\n", 1)[0]
session_id: str | None = None
for line in header_block.splitlines():
    if line.lower().startswith("mcp-session-id:"):
        session_id = line.split(":", 1)[1].strip()
        break

print(f"Session id: {session_id}")


### 2.2 Send `notifications/initialized`

This is a standard follow-up message after `initialize`.

In [ ]:
notif_msg: dict[str, object] = {
    "jsonrpc": "2.0",
    "method": "notifications/initialized",
}

payload_text: str = json.dumps(notif_msg, ensure_ascii=False)

session_header_args: list[str] = ["-H", f"Mcp-Session-Id: {session_id}"] if session_id else []

cmd: list[str] = [
    "curl",
    "-sS",
    "-i",
    "-X",
    "POST",
    MCP_ENDPOINT,
    "-H",
    "Accept: application/json, text/event-stream",
    "-H",
    "Content-Type: application/json",
    "-H",
    f"MCP-Protocol-Version: {MCP_PROTOCOL_VERSION}",
    "-H",
    f"Origin: {BASE_URL}",
    *session_header_args,
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
print(subprocess.check_output(cmd, input=payload_text, text=True)[:2000])


### 2.3 List available tools

The response contains the available MCP tools and their input schemas.

In [ ]:
tools_list_msg: dict[str, object] = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list",
    "params": {},
}

payload_text: str = json.dumps(tools_list_msg, ensure_ascii=False)
session_header_args: list[str] = ["-H", f"Mcp-Session-Id: {session_id}"] if session_id else []

cmd: list[str] = [
    "curl",
    "-sS",
    "-X",
    "POST",
    MCP_ENDPOINT,
    "-H",
    "Accept: application/json, text/event-stream",
    "-H",
    "Content-Type: application/json",
    "-H",
    f"MCP-Protocol-Version: {MCP_PROTOCOL_VERSION}",
    "-H",
    f"Origin: {BASE_URL}",
    *session_header_args,
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
out: str = subprocess.check_output(cmd, input=payload_text, text=True)

tools_payload: object = json.loads(out)
if not isinstance(tools_payload, dict):
    raise ValueError("Expected a JSON object from tools/list.")

print(json.dumps(tools_payload, ensure_ascii=False, indent=2, sort_keys=True))


### 2.4 Call `create_llamator_run`

This tool submits a run and waits for completion.

The output is a machine-readable JSON structure that includes:
- `job_id`
- `aggregated`
- `artifacts_download_url` (optional)
- `error_notice` (optional)

In [ ]:
mcp_create_msg: dict[str, object] = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "create_llamator_run",
        "arguments": {
            "req": payload_preset,
        },
    },
}

payload_text: str = json.dumps(mcp_create_msg, ensure_ascii=False)
session_header_args: list[str] = ["-H", f"Mcp-Session-Id: {session_id}"] if session_id else []

cmd: list[str] = [
    "curl",
    "-sS",
    "-X",
    "POST",
    MCP_ENDPOINT,
    "-H",
    "Accept: application/json, text/event-stream",
    "-H",
    "Content-Type: application/json",
    "-H",
    f"MCP-Protocol-Version: {MCP_PROTOCOL_VERSION}",
    "-H",
    f"Origin: {BASE_URL}",
    *session_header_args,
    *AUTH_HEADER_ARGS,
    "--data-binary",
    "@-",
]

print("Running:", " ".join(cmd))
out: str = subprocess.check_output(cmd, input=payload_text, text=True)

mcp_created_payload: object = json.loads(out)
if not isinstance(mcp_created_payload, dict):
    raise ValueError("Expected a JSON object from tools/call.")

print(json.dumps(mcp_created_payload, ensure_ascii=False, indent=2, sort_keys=True))